## Logging functions

São funções utilizadas para registrar (logar) informações relevantes durante o treinamento e avaliação de modelos de aprendizado de máquina. Essas informações incluem métricas, parâmetros, artefatos (como gráficos ou arquivos) e modelos, permitindo rastrear experimentos e facilitar reprodutibilidade.

1. As duas primeiras são :
    * Uri de rastramento de conj. de pontos do MLflow e Uri de rastramento de ponto do MLflow

    ```python
    mlflow.set_tracking_uri()
    ```
    > É usada para definir o local de sua escolha, onde deseja manter os rastros de seu codigo
    > A função de rastreamento definidda no mlflow é usada para definir o servidor de rastreamento padrão 
    > Se deseja alterar o local pode fazer por meio dessa função

    ```python
    mlflow.get_tracking_uri()
    ```
    > É recuperar o caminho da localização
exemplo abaixo:

In [3]:
import mlflow

In [4]:
mlflow.set_tracking_uri(uri="https://localhost:5000/home")

In [5]:
print(mlflow.get_tracking_uri())

https://localhost:5000/home


In [6]:
%%writefile logg.py
import warnings
import argparse
import logging

# --------
import pandas as pd
import numpy as np

#----------
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

#--------- Importando mlflow
import mlflow
import mlflow.sklearn as mlsk

logging.basicConfig(
    level=logging.WARN
)
logger = logging.getLogger(__name__)

"""
e é uma biblioteca do Python que facilita a criação de interfaces de 
linha de comando (CLI) para seus scripts.  Ele permite que você defina 
quais argumentos (ou parâmetros) o script pode receber e os processa 
automaticamente para que você os utilize dentro do programa.
"""
parser = argparse.ArgumentParser()
parser.add_argument(
    "--alpha",
    type = float,
    required = False,
    default = 0.5
)
parser.add_argument(
    "--l1_ratio",
    type = float,
    required = False,
    default = 0.5
)
args = parser.parse_args()


# Metricas
def metricas(ytrue, ypred):
    rmse = root_mean_squared_error(ytrue, ypred)
    mae  = mean_absolute_error(ytrue, ypred)
    r2_  = r2_score(ytrue, ypred)

    return rmse, mae, r2_


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    url = (
        "https://raw.githubusercontent.com/shrikant-temburwar/Wine-Quality-Dataset/master/winequality-red.csv"
    )

    try:
        data_ = pd.read_csv(url, sep=';')

    except Exception as e:
        logger.exception(
            "OBS: Algo deu de errado", 0
        )
    train, test = train_test_split(data_)

    xtrain, ytrain = (
        train.drop(columns=["quality"]),
        train["quality"]
    )

    xtest, ytest = (
        test.drop(columns=["quality"]),
        test["quality"]
    )

    alpha = args.alpha
    l1_ratio = args.l1_ratio
    # Definindo o URI
    mlflow.set_tracking_uri(
        uri="./mytracks"
    )
    print("Uri :", mlflow.get_tracking_uri())

    # Definindo um nome do experimento
    exp = mlflow.set_experiment(experiment_name="ExperimentRegElastic")

    with mlflow.start_run(experiment_id=exp.experiment_id):
        elastic = ElasticNet(
            alpha=alpha, l1_ratio=l1_ratio, random_state=32
        )
        elastic.fit(xtrain, ytrain)
    
        pred_ = elastic.predict(xtest)
    
        
        rmse, mae, r2 = metricas(ytest, pred_)
        
        print(elastic.get_params)
        print("RMSE :",rmse)
        print("MAE  :",mae)
        print("R2   :",r2)

        # Logando os paramatros do modelos
        mlflow.log_params(
            {"alpha":alpha, "l1_ratio":l1_ratio}
        )

        # Logando metricas
        mlflow.log_metrics(
            dict(
                root_mean_squared_error_ = rmse,
                mean_absolute_error_     = mae,
                r2_score_                = r2
            )
        )

        # Rastreando modelo treinado
        sign = mlflow.models.infer_signature(
            xtrain, pred_
        )
        mlsk.log_model(
            elastic, "modelo_elasticNet", signature=sign
        )

Overwriting logg.py


```bash
 mlflow ui --backend-store-uri ./mytracks 
```

### Criando experimentos

```python
# Cria um novo experimento 
mlflow.create_experiment() # ela retorna o id do experimento
```
Parametros:
* `name` = defini o nome experimento **deve ser único**

* `artifact_location` = defini o local dos artefartos onde ficará armazenados . **Opicional**
*  `tags` =  mais adiante


```python
# Defini um novo experimento já existente
mlflow.set_experiment()
```


In [11]:
%%writefile logg2_createExperiment2.py
import warnings
import argparse
import logging

# --------
import mlflow.exceptions
import pandas as pd
import numpy as np

#----------
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

#--------- Importando mlflow
import mlflow
import mlflow.sklearn as mlsk

logging.basicConfig(
    level=logging.WARN
)
logger = logging.getLogger(__name__)

"""
e é uma biblioteca do Python que facilita a criação de interfaces de 
linha de comando (CLI) para seus scripts.  Ele permite que você defina 
quais argumentos (ou parâmetros) o script pode receber e os processa 
automaticamente para que você os utilize dentro do programa.
"""
parser = argparse.ArgumentParser()
parser.add_argument(
    "--alpha",
    type = float,
    required = False,
    default = 0.5
)
parser.add_argument(
    "--l1_ratio",
    type = float,
    required = False,
    default = 0.5
)
args = parser.parse_args()


# Metricas
def metricas(ytrue, ypred):
    rmse = root_mean_squared_error(ytrue, ypred)
    mae  = mean_absolute_error(ytrue, ypred)
    r2_  = r2_score(ytrue, ypred)

    return rmse, mae, r2_


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    url = (
        "https://raw.githubusercontent.com/shrikant-temburwar/Wine-Quality-Dataset/master/winequality-red.csv"
    )

    try:
        data_ = pd.read_csv(url, sep=';')

    except Exception as e:
        logger.exception(
            "OBS: Algo deu de errado", 0
        )
    train, test = train_test_split(data_)

    xtrain, ytrain = (
        train.drop(columns=["quality"]),
        train["quality"]
    )

    xtest, ytest = (
        test.drop(columns=["quality"]),
        test["quality"]
    )

    alpha = args.alpha
    l1_ratio = args.l1_ratio
    # Definindo o URI
    mlflow.set_tracking_uri(
        uri="./mytracks"
    )
    print("Uri :", mlflow.get_tracking_uri())

    # Definindo um nome do experimento
    try:
        exp_id = mlflow.create_experiment(
        name="Elastic_",
        tags=dict(version_= "1.0.0")
        )
    
        get_ = mlflow.get_experiment(experiment_id=exp_id)
    except mlflow.exceptions.MlflowException:
        get_ = mlflow.set_experiment(experiment_name="Elastic_")

    str_ = f"""
Name..............: {get_.name}
Id................: {get_.experiment_id}
Tag...............: {get_.tags}
LyfeCycle_Stage...: {get_.lifecycle_stage}
Artifact Location : {get_.artifact_location}
Timestamp Create..: {get_.creation_time}
"""
    print(str_)
    with mlflow.start_run(experiment_id=get_.experiment_id):
        elastic = ElasticNet(
            alpha=alpha, l1_ratio=l1_ratio, random_state=32
        )
        elastic.fit(xtrain, ytrain)
    
        pred_ = elastic.predict(xtest)
    
        
        rmse, mae, r2 = metricas(ytest, pred_)
        
        print(elastic.get_params)
        print("RMSE :",rmse)
        print("MAE  :",mae)
        print("R2   :",r2)

        # Logando os paramatros do modelos
        mlflow.log_params(
            {"alpha":alpha, "l1_ratio":l1_ratio}
        )

        # Logando metricas
        mlflow.log_metrics(
            dict(
                root_mean_squared_error_ = rmse,
                mean_absolute_error_     = mae,
                r2_score_                = r2
            )
        )

        # Rastreando modelo treinado
        sign = mlflow.models.infer_signature(
            xtrain, pred_
        )
        mlsk.log_model(
            elastic, "modelo_elasticNet", signature=sign
        )

Writing logg2_createExperiment2.py


```python
#utilize se existir um experiemnto para rodar 
set_experiment()
```
parametros:
* `experiment_name` : coloque o nome do experimento que já foi criado
* `experiment_id`: mesma ideia do experiment_name

> Retorna uma instancia do experimento de entidades do mlflow que representa um novo experimento ativo

In [13]:
%%writefile config.yml
parameters:
    loss: "squared_error"
    learning_rate: 0.1
    n_estimators : 50
    max_features : "sqrt"

Writing config.yml


In [24]:
%%writefile logg2_createExperiment3.py
import mlflow.models
import mlflow.models
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.metrics  import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import omegaconf

import mlflow
import mlflow.sklearn



def train_(X, y):

    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=40)
    mlflow.set_tracking_uri(
        uri="./mytracks"
    )
    print("Uri :", mlflow.get_tracking_uri())
    try:
        exp_ = mlflow.create_experiment(
            name = "GradientBoss",
            tags = dict(version_ = "1.0.0")
        )
        get_ = mlflow.get_experiment(experiment_id=exp_)
    except mlflow.exceptions.MlflowException as e:
        get_ = mlflow.set_experiment(experiment_name="GradientBoss")

    str_ = f"""
Name..............: {get_.name}
Id................: {get_.experiment_id}
Tag...............: {get_.tags}
LyfeCycle_Stage...: {get_.lifecycle_stage}
Artifact Location : {get_.artifact_location}
Timestamp Create..: {get_.creation_time}
"""
    print(str_)
    #mlflow.start_run(experiment_id=get_.experiment_id, run_name="run_1"):
    with mlflow.start_run(run_id="c0ac8f8bcb594410a03d9bfe33697368"):
        # Carregando a config.yml
        config = omegaconf.OmegaConf.load(file_="config.yml")
        
        boost_ = GradientBoostingRegressor(**config.parameters)
        boost_.fit(xtrain, ytrain)

        # Log Metrics
        mlflow.log_metrics(
            dict(
                mean_squared_error_ = mean_squared_error(ytrain, boost_.predict(xtrain)),
                r2_score_           = r2_score(ytrain, boost_.predict(xtrain))
            )
        )

        # Logando parametros do modelo
        mlflow.log_params(
            config.parameters
        )

        # Carreando modelo

        sign = mlflow.models.infer_signature(
            xtrain, boost_.predict(xtrain)
        )
        mlflow.sklearn.log_model(
             boost_,"GradientBoostingRegressor_2", signature=sign
        )
if __name__ == "__main__":
    X, y = fetch_california_housing(return_X_y=True)
    train_(X, y)

Overwriting logg2_createExperiment3.py


In [ ]:
%%writefile logg2_createExperiment4.py
import mlflow.models
import mlflow.models
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.metrics  import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import omegaconf

import mlflow
import mlflow.sklearn



def train_(X, y):

    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=40)
    mlflow.set_tracking_uri(
        uri="./mytracks"
    )
    print("Uri :", mlflow.get_tracking_uri())
    try:
        exp_ = mlflow.create_experiment(
            name = "GradientBoss",
            tags = dict(version_ = "1.0.0")
        )
        get_ = mlflow.get_experiment(experiment_id=exp_)
    except mlflow.exceptions.MlflowException as e:
        get_ = mlflow.set_experiment(experiment_name="GradientBoss")

    str_ = f"""
Name..............: {get_.name}
Id................: {get_.experiment_id}
Tag...............: {get_.tags}
LyfeCycle_Stage...: {get_.lifecycle_stage}
Artifact Location : {get_.artifact_location}
Timestamp Create..: {get_.creation_time}
"""
    print(str_)
    #mlflow.start_run(experiment_id=get_.experiment_id, run_name="run_1"):
    mlflow.start_run(run_id="c0ac8f8bcb594410a03d9bfe33697368")
    # Carregando a config.yml
    config = omegaconf.OmegaConf.load(file_="config.yml")
    
    boost_ = GradientBoostingRegressor(**config.parameters)
    boost_.fit(xtrain, ytrain)
    # Log Metrics
    mlflow.log_metrics(
        dict(
            mean_squared_error_ = mean_squared_error(ytrain, boost_.predict(xtrain)),
            r2_score_           = r2_score(ytrain, boost_.predict(xtrain))
        )
    )
    # Logando parametros do modelo
    mlflow.log_params(
        config.parameters
    )
    # Carreando modelo
    sign = mlflow.models.infer_signature(
        xtrain, boost_.predict(xtrain)
    )
    mlflow.sklearn.log_model(
         boost_,"GradientBoostingRegressor_2", signature=sign
    )
    mlflow.end_run(status="finalizado")

if __name__ == "__main__":
    X, y = fetch_california_housing(return_X_y=True)
    train_(X, y)

```python
# Inicia nova entre 
mlflow.active_run()

# Inicia a ultima run
mlflow.last_active_run()
```

In [2]:
%%writefile logg2_createExperiment5.py
import mlflow.models
import mlflow.models
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.metrics  import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import omegaconf

import mlflow
import mlflow.sklearn



def train_(X, y):

    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=40)
    mlflow.set_tracking_uri(
        uri="./mytracks"
    )
    print("Uri :", mlflow.get_tracking_uri())
    try:
        exp_ = mlflow.create_experiment(
            name = "GradientBoss",
            tags = dict(version_ = "1.0.0")
        )
        get_ = mlflow.get_experiment(experiment_id=exp_)
    except mlflow.exceptions.MlflowException as e:
        get_ = mlflow.set_experiment(experiment_name="GradientBoss")

    str_ = f"""
Name..............: {get_.name}
Id................: {get_.experiment_id}
Tag...............: {get_.tags}
LyfeCycle_Stage...: {get_.lifecycle_stage}
Artifact Location : {get_.artifact_location}
Timestamp Create..: {get_.creation_time}
"""
    print(str_)

    mlflow.start_run()
    # Carregando a config.yml
    config = omegaconf.OmegaConf.load(file_="config.yml")
    
    boost_ = GradientBoostingRegressor(**config.parameters)
    boost_.fit(xtrain, ytrain)
    # Log Metrics
    mlflow.log_metrics(
        dict(
            mean_squared_error_ = mean_squared_error(ytrain, boost_.predict(xtrain)),
            r2_score_           = r2_score(ytrain, boost_.predict(xtrain))
        )
    )
    # Logando parametros do modelo
    mlflow.log_params(
        config.parameters
    )
    # Carreando modelo
    sign = mlflow.models.infer_signature(
        xtrain, boost_.predict(xtrain)
    )
    mlflow.sklearn.log_model(
         boost_,"GradientBoostingRegressor_2", signature=sign
    )


    run_ = mlflow.last_active_run()
    print("active_run_id :", run_.info.run_id)
    print("active_run_name :", run_.info.run_name)

    mlflow.end_run()

if __name__ == "__main__":
    X, y = fetch_california_housing(return_X_y=True)
    train_(X, y)

Overwriting logg2_createExperiment5.py


> Pode retirar o ```with```
> Iniciar start_run e terminar com o end_run

``` python
# Log/registra dos parametros do modelo( melhor usar este)
mlflow.log_params()

# Log/registra metricas 
mlflow.log_metrics()

# log/registra multiplosartifacts
mlflow.log_artifacts()
```